In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.

    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


# let's also shuffle the data

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 300

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

# batch the test data
test_data = test_data.batch(num_test_samples)


# takes next batch (it is the only batch)
# because as_supervized=True, we've got a 2-tuple structure
validation_inputs, validation_targets = next(iter(validation_data))

2024-06-18 16:17:56.445162: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [3]:
input_size = 784
output_size = 10

hidden_layer_size = 100
    

model = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer

    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    #tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    #tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    #tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'), 
    #tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
    
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

/Users/paritp/anaconda3/envs/py3-TF2.0/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [4]:
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
# determine the maximum number of epochs
NUM_EPOCHS = 5

early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2)

# we fit the model, specifying the
# training data
# the total number of epochs
# and the validation data we just created ourselves in the format: (inputs,targets)
model.fit(train_data, epochs=NUM_EPOCHS, callbacks = [early_stopping], validation_data=(validation_inputs, validation_targets), verbose =2)

Epoch 1/5
180/180 - 2s - 8ms/step - accuracy: 0.8778 - loss: 0.4424 - val_accuracy: 0.9463 - val_loss: 0.1809
Epoch 2/5
180/180 - 1s - 4ms/step - accuracy: 0.9534 - loss: 0.1557 - val_accuracy: 0.9643 - val_loss: 0.1172
Epoch 3/5
180/180 - 1s - 4ms/step - accuracy: 0.9679 - loss: 0.1094 - val_accuracy: 0.9720 - val_loss: 0.0938
Epoch 4/5
180/180 - 1s - 4ms/step - accuracy: 0.9749 - loss: 0.0840 - val_accuracy: 0.9750 - val_loss: 0.0797
Epoch 5/5
180/180 - 1s - 4ms/step - accuracy: 0.9804 - loss: 0.0660 - val_accuracy: 0.9815 - val_loss: 0.0606


In [6]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.9740 - loss: 0.0791
Test loss: 0.08. Test accuracy: 97.40%
